In [15]:
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import itertools as it
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import mutual_info_regression as mir
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder as onehot
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from statsmodels.tsa.deterministic import DeterministicProcess
from sklearn.metrics import mean_absolute_percentage_error as mape
from statsmodels.tsa.stattools import pacf
import xgboost as xgb
from itertools import product
from scipy import signal
from scipy import stats
from statsmodels.tsa.deterministic import Fourier
from sklearn.model_selection import TimeSeriesSplit

In [16]:
de_train_final = pd.read_csv('./data/de_train_final.csv')
y_de_train = pd.read_csv('./data/y_de_train.csv')
fr_train_final = pd.read_csv('./data/fr_train_final.csv')
y_fr_train = pd.read_csv('./data/y_fr_train.csv')

de_test_final = pd.read_csv('./data/de_test_final.csv')
fr_test_final = pd.read_csv('./data/fr_test_final.csv')

In [17]:
y_de_train.columns


Index(['TARGET'], dtype='object')

In [18]:

print("de_train_final with shape:", de_train_final.shape)
print("y_de_train with shape:", y_de_train.shape)
print("fr_train_final with shape:", fr_train_final.shape)
print("y_fr_train with shape:", y_fr_train.shape)
print("de_test_final with shape:", de_test_final.shape)
print("fr_test_final with shape:", fr_test_final.shape)

de_train_final with shape: (643, 35)
y_de_train with shape: (643, 1)
fr_train_final with shape: (851, 35)
y_fr_train with shape: (851, 1)
de_test_final with shape: (289, 35)
fr_test_final with shape: (365, 35)


In [19]:
de_train_final.columns

Index(['ID', 'DE_CONSUMPTION', 'DE_FR_EXCHANGE', 'DE_NET_EXPORT', 'DE_GAS',
       'DE_COAL', 'DE_HYDRO', 'DE_NUCLEAR', 'DE_SOLAR', 'DE_WINDPOW',
       'DE_LIGNITE', 'DE_RESIDUAL_LOAD', 'DE_RAIN', 'DE_WIND', 'DE_TEMP',
       'GAS_RET', 'COAL_RET', 'CARBON_RET', 'DE_CONSUMPTION_prev_day',
       'DE_FR_EXCHANGE_prev_day', 'DE_NET_EXPORT_prev_day', 'DE_GAS_prev_day',
       'DE_COAL_prev_day', 'DE_HYDRO_prev_day', 'DE_NUCLEAR_prev_day',
       'DE_SOLAR_prev_day', 'DE_WINDPOW_prev_day', 'DE_LIGNITE_prev_day',
       'DE_RESIDUAL_LOAD_prev_day', 'DE_RAIN_prev_day', 'DE_WIND_prev_day',
       'DE_TEMP_prev_day', 'GAS_RET_prev_day', 'COAL_RET_prev_day',
       'CARBON_RET_prev_day'],
      dtype='object')

In [20]:
# Assuming 'TARGET' is your target variable
X = de_train_final
y = y_de_train

# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

# Train the model
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(x_test)

# Calculate Spearman Correlation

spearman_corr_lr_value = spearman_corr_lr.item()

# 使用f-string格式化输出
print(f"Spearman Correlation for Linear Regression: {spearman_corr_lr_value:.1%}")

Spearman Correlation for Linear Regression: 71.0%


In [21]:
de_train_pred = lr_model.predict(X)
de_train = de_train_final.reset_index()
de_train['TARGET'] = de_train_pred
de_train = de_train[['ID', 'TARGET']]

de_test = lr_model.predict(de_test_final)
de_test_pred = de_test_final.reset_index()
de_test_pred['TARGET'] = de_test
de_test_pred = de_test_pred[['ID', 'TARGET']]

In [22]:
de_test = lr_model.predict(de_test_final)
de_test_pred = de_test_final.reset_index()
de_test_pred['TARGET'] = de_test
de_test_pred = de_test_pred[['ID', 'TARGET']]

In [23]:
# Assuming 'TARGET' is your target variable
X = fr_train_final
y = y_fr_train

# Split the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

# Train the model

lr_model.fit(x_train, y_train)

# Predict on the test set
y_pred_lr = lr_model.predict(x_test)

# Calculate Spearman Correlation
spearman_corr_lr = spearmanr(y_pred_lr, y_test).correlation
print(f"Spearman Correlation for Linear Regression: {spearman_corr_lr:.1%}")

Spearman Correlation for Linear Regression: 20.6%


In [24]:
fr_train_pred = lr_model.predict(X)
fr_result = fr_train_final.reset_index()
fr_result['TARGET'] = fr_train_pred
fr_result = fr_result[['ID', 'TARGET']]

fr_test = lr_model.predict(fr_test_final)
fr_test_pred = fr_test_final.reset_index()
fr_test_pred['TARGET'] = fr_test
fr_test_pred = fr_test_pred[['ID', 'TARGET']]

In [25]:
y_train = pd.read_csv('../challenge_data/Y_train.csv')
train_pred = pd.DataFrame()
train_pred['ID'] = y_train['ID']

train_pred = train_pred.merge(de_train[['ID', 'TARGET']], on='ID', how='left')
train_pred = train_pred.merge(fr_result[['ID', 'TARGET']], on='ID', how='left')

train_pred['TARGET'] = train_pred['TARGET_x'].combine_first(train_pred['TARGET_y'])

train_pred = train_pred.drop(['TARGET_x', 'TARGET_y'], axis=1)

spearman_corr_lr = spearmanr(train_pred['TARGET'], y_train['TARGET']).correlation
print(f"Spearman Correlation for Linear Regression: {spearman_corr_lr:.1%}")

Spearman Correlation for Linear Regression: 51.5%


In [26]:
df_test = pd.read_csv('../challenge_data/X_test.csv')
test_pred = pd.DataFrame()
test_pred['ID'] = df_test['ID']

test_pred = test_pred.merge(de_test_pred[['ID', 'TARGET']], on='ID', how='left')
test_pred = test_pred.merge(fr_test_pred[['ID', 'TARGET']], on='ID', how='left')
test_pred['TARGET'] = test_pred['TARGET_x'].combine_first(test_pred['TARGET_y'])
test_pred = test_pred.drop(['TARGET_x', 'TARGET_y'], axis=1)

In [27]:
test_pred

,ID,TARGET
0,1115,0.056517
1,1202,0.027124
2,1194,0.263040
3,1084,0.310935
4,1135,0.144030
...,...,...
649,879,-0.928058
650,673,-0.646584
651,1641,0.285855
652,712,2.176437


In [28]:
test_pred.to_csv('./Submission/' + 'MA_test' + '.csv', index=False)